<a href="https://colab.research.google.com/github/raz0208/Natural-Language-Processing-Practices/blob/main/TopicModelling/NLP_TopicsModellingPractices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modelling Practices in NLP


1. Semantic Signal Separation
2. KeyNMF
3. ClusteringTopicModel



### 1. Semantic Signal Separation

In [31]:
# Install turftopic libraries
!pip install turftopic
!pip install datasets

In [32]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from turftopic import SemanticSignalSeparation
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

In [33]:
# Read and load dataset
ds = pd.read_csv('political_ideologies_train.csv')
texts = ds["statement"]

print(texts)

0       Climate change, and the escalating environment...
1       I believe in the foundational importance of th...
2       I firmly believe that the principle of separat...
3       I firmly believe in the separation of church a...
4       I firmly believe in the power of free markets ...
                              ...                        
2555    I believe in the power of free markets to driv...
2556    I firmly believe in the traditional family str...
2557    Every individual, regardless of their gender, ...
2558    I firmly believe in the significance of religi...
2559    I firmly believe in the principle of individua...
Name: statement, Length: 2560, dtype: object


In [34]:
encoder = SentenceTransformer('paraphrase-MiniLM-L12-v2')
embeddings = encoder.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

In [39]:
print(embeddings)

[[-0.20092632  0.33961082  0.02055947 ... -0.14055775 -0.09066175
  -0.05947793]
 [-0.05251725  0.2111203  -0.2350792  ... -0.19393641  0.06882618
  -0.02158776]
 [-0.06461112  0.07165432 -0.08635489 ... -0.05988945  0.02278206
   0.0677686 ]
 ...
 [ 0.2636646   0.07064892 -0.10288888 ...  0.00702172  0.0945406
  -0.02747649]
 [ 0.2114343   0.13071813 -0.26949868 ... -0.00532549 -0.03988217
  -0.08325328]
 [-0.3067356   0.26985478 -0.18770279 ... -0.16361783  0.10027055
  -0.09515021]]


In [36]:
model = SemanticSignalSeparation(4, encoder=encoder, random_state=42)
doc_topic_matrix = model.fit_transform(texts, embeddings=embeddings)

Output()

/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning:

FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.

[12:58:24] Decomposition done.                                                                        ]8;id=727742;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=628850;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py#144\144]8;;\

           Term extraction done.                                                                      ]8;id=86792;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=894888;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py#147\147]8;;\

[12:58:38] Vocabulary encoded.                                                                        ]8;id=923308;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=136453;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py#157\157]8;;\

           Model fitting done.                                                                        ]8;id=369043;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=494239;file:///usr/local/lib/python3.11/dist-packages/turftopic/models/decomp.py#170\170]8;;\

In [37]:
model.print_topics(top_k=10)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic ID ┃ Highest Ranking                                  ┃ Lowest Ranking                                    ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        0 │ warming, carbon, environment, solar, change,     │ wealth, taxation, prosperity, wealthiest, tax,    │
│          │ planet, greenhouse, environmental, shift, fossil │ profit, taxes, entrepreneurship, fiscal,          │
│          │                                                  │ government                                        │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        1 │ religious, religion, church, religions, faith,   │ spending, fiscal, budget, economic, economies,    │
│          │ faiths, respectful, respect, distinct, lgbtq     │ economy, growth, investments, economically,       │
│          │                                                  │ productive                                        │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        2 │ marriage, structure, traditions, relationships,  │ healthcare, medical, patient, health, care,       │
│          │ structures, tradition, tensions, conflicts,      │ healthy, treated, healthier, treatment,           │
│          │ conflict, couples                                │ treatments                                        │
├──────────┼──────────────────────────────────────────────────┼───────────────────────────────────────────────────┤
│        3 │ freedoms, liberties, iran, freedom, faith,       │ households, adoption, racial, household, racism,  │
│          │ religion, protection, secure, military, security │ families, ethnicity, mothers, childcare, races    │
└──────────┴──────────────────────────────────────────────────┴───────────────────────────────────────────────────┘

In [38]:
model.plot_concept_compass(0, 1)